# Error and Exceptions

* Syntax errors: typos, wrong intendations, etc.

In [1]:
for i in range(0):
print(i)

IndentationError: expected an indented block (<ipython-input-1-8f07eeeb568f>, line 2)

In [2]:
print 1

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(1)? (<ipython-input-2-c94594b6b28f>, line 1)

* Exceptions: error raised when attempting an execution

In [3]:
print(1/0)

ZeroDivisionError: division by zero

In [4]:
print(animal)

NameError: name 'animal' is not defined

In [5]:
print(1 + 'abc')

TypeError: unsupported operand type(s) for +: 'int' and 'str'

### Handling exceptions

```
try:
    
    Statement 1
    
    Statement 2
    
    ...
    

except Exception:

    Handling the exception with some statement

```

In [86]:
# Concatenate several strings into 1

def concatenate(*args):
    
    string = ''
    
    for arg in args:
        
        string += arg
    
    return string

In [87]:
concatenate('Alpha and ', 'Bravo and ', 'Charlie')

'Alpha and Bravo and Charlie'

But if someone want to concatenate a number

In [88]:
concatenate('Alpha and ', 'Bravo and ', 'Charlie and', 123)

TypeError: must be str, not int

We have got a TypeError exception, which we would like to fix.

In [89]:
# add error handling for this function

def concatenate(*args):
    
    string = ''
    
    for arg in args:
        
        try:
            
            string += arg
        
        except Exception:
            
            # convert it to a string
            string += str(arg)
            
    
    return string

In [90]:
concatenate('Alpha and ', 'Bravo and ', 'Charlie')

'Alpha and Bravo and Charlie'

In [91]:
concatenate('Alpha and ', 'Bravo and ', 'Charlie and ', 123)

'Alpha and Bravo and Charlie and 123'

In [92]:
concatenate('Alpha and ', 'Bravo and ', 'Charlie and ', None)

'Alpha and Bravo and Charlie and None'

Having 'None' doesn't look good to me. I want it to be excluded if a NULL is passed to these positional arguments.

In [104]:
# add error handling for this function

def concatenate(*args):
    
    string = ''
    
    for arg in args:
        
        try:
            
            if arg:
                string += arg
            
            else:
                raise ValueError # Let's make None an invalid value here
        
        except TypeError:
            
            # convert it to a string
            string += str(arg)
        
        except ValueError:
            
            # ignore value errors
            pass
            
    
    return string

In [105]:
concatenate('Alpha and ', 'Bravo and ', 'Charlie and ', None)

'Alpha and Bravo and Charlie and '

### Defining exceptions

We have seen those built-in exceptions TypeError, NameError, ValueError, etc. These are all inheritance from Exception.

In [106]:
for exceptionType in [TypeError, NameError, ValueError]:
    
    print(issubclass(exceptionType, Exception))

True
True
True


We are also allowed to define our own exception. Let's look at an example. 

In [126]:
class Animal(object):
    
    def __init__(self, _type, _isPet = True):
        
        self.type = _type
        self.isPet = _isPet
        
    
    def adopt(self, name = None):
        
        self.name = name if name else 'Your %s' % self.type
        
        print('%s is adopted' % self.name)
        
        
        

dog = Animal('Dog', _isPet = True)
cat = Animal('Cat', _isPet = True)
tiger = Animal('Tiger', _isPet = False)


Let's try adopt all these animals!

In [127]:
def adoptManyAnimals(*animals):
    
    for animal in animals:
        animal.adopt()

In [128]:
adoptManyAnimals(dog, cat, tiger)

Your Dog is adopted
Your Cat is adopted
Your Tiger is adopted


Wait... you probably don't want to adopt a tiger! Let's stop people from doing so. Here we are raising an exception
```
Exception('This animal is not a pet')
```

Recap: This is an *instance* of the Exception class.

In [129]:
class Animal(object):
    
    def __init__(self, _type, _isPet = True):
        
        self.type = _type
        self.isPet = _isPet
        
    
    def adopt(self, name = None):
        
        # if the animal is not supposed to be a pet, raise an exception!
        if not self.isPet:
            raise Exception('This animal is not a pet')
        
        self.name = name if name else 'Your %s' % self.type
        
        print('%s is adopted' % self.name)
        
        
        

dog = Animal('Dog', _isPet = True)
cat = Animal('Cat', _isPet = True)
tiger = Animal('Tiger', _isPet = False)

In [130]:
adoptManyAnimals(dog, cat, tiger)

Your Dog is adopted
Your Cat is adopted


Exception: This animal is not a pet

Now let's handle this exception.

In [131]:
def adoptManyAnimals(*animals):
    
    for animal in animals:
        
        try:
            animal.adopt()
        
        except Exception as err: # as err: give a nick name to the caught exception because we want to print it out
            
            # print the message
            print('%s is not adopted because %s' % (animal.type, err))
            # ignore this animal
            
            
            

In [132]:
adoptManyAnimals(dog, cat, tiger)

Your Dog is adopted
Your Cat is adopted
Tiger is not adopted because This animal is not a pet


Now it is working perfect. Wait, someone just abused our programme:

In [133]:
adoptManyAnimals(dog, cat, tiger, 123)

Your Dog is adopted
Your Cat is adopted
Tiger is not adopted because This animal is not a pet


AttributeError: 'int' object has no attribute 'type'

This error message is misleading. The problem is we assumed all exceptions come from the isPet issue. But essentially here we are passing an integer as an argument which doesn't have the method `adopt` (what does it even mean to adopt an integer??) To fix this, let's carefully design our exception. Instead of raising an instance of Exception (`Exception('This animal is not a pet')`), let's create an inherited class so that it behaves just like TypeError, NameError, etc.

In [137]:
class PetError(Exception):
    
    """Exception raised when a non-pet gets adopted.

    Attributes:
        expression -- input expression in which the error occurred
        message -- explanation of the error
    """
    
    def __init__(self):
        pass

In [138]:
class Animal(object):
    
    def __init__(self, _type, _isPet = True):
        
        self.type = _type
        self.isPet = _isPet
        
    
    def adopt(self, name = None):
        
        # if the animal is not supposed to be a pet, raise an exception!
        if not self.isPet:
            
            # raise our PetError instead
            raise PetError
        
        self.name = name if name else 'Your %s' % self.type
        
        print('%s is adopted' % self.name)
        
        
        

dog = Animal('Dog', _isPet = True)
cat = Animal('Cat', _isPet = True)
tiger = Animal('Tiger', _isPet = False)

In [139]:
def adoptManyAnimals(*animals):
    
    for animal in animals:
        
        try:
            animal.adopt()
        
        except PetError as err: # as err: give a nick name to the caught exception because we want to print it out
            
            # print the message
            print('%s is not adopted because %s' % (animal.type, err))
            # ignore this animal
        
        except Exception as err:
            print(err)

In [140]:
adoptManyAnimals(dog, cat, tiger, 123)

Your Dog is adopted
Your Cat is adopted
Tiger is not adopted because 
'int' object has no attribute 'adopt'


Now the errors are classified and clearly labelled - much easier to read! A bonus question - can we print a line to tell which animal is the next one on the list?

If a `finally` clause is present, it will execute as the last task before the `try` statement completes. It runs whether or not the `try` statement produces an exception.

In [142]:
def adoptManyAnimals(*animals):
    
    for animal in animals:
        
        try:
            animal.adopt()
        
        except PetError as err: # as err: give a nick name to the caught exception because we want to print it out
            
            # print the message
            print('%s is not adopted because %s' % (animal.type, err))
            # ignore this animal
        
        except Exception as err:
            print(err)
        
        finally:
            print('Moving on to the next one')

In [143]:
adoptManyAnimals(dog, cat, tiger, 123)

Your Dog is adopted
Moving on to the next one
Your Cat is adopted
Moving on to the next one
Tiger is not adopted because 
Moving on to the next one
'int' object has no attribute 'adopt'
Moving on to the next one
